# Project 4: Natural Language Processing 
---

**Group 9: Aidan Stocks, Hugo Reinicke, Nicola Clark, Jonas-Mika Senghaas**

Submission: *03.06.2021* / Last Modified: *02.06.2021*

---

This notebook contains the step-by-step process of building a natural language machine learning model to automatically detect *(a)* Hatespeech and *(b)* Emotion in tweets on the social network Twitter. 

The initial data was obtained from the [TweetEval](https://github.com/cardiffnlp/tweeteval#evaluating-your-system) GitHub repository, that provides data for supervised training of classifiers for natural language processing, more specifically it provides prepared data for several mini-project involving the analysis of different characteristics of tweets.

## Introduction
---
Social media is omnipresent in today's world. We use messengers to communicate, share pictures, music, thoughts - in short - our life on the internet with people that are close, and maybe also not as close to us. Twitter is one of those social networks. The american social networking service allows its users to post and interact with messages known as through so-called tweets. 280 character postings on the online-service that can be liked, commented, threaded and shared. Since its launch in 2006, *Twitter* has grown massively, nowadays reporting hundreds of million of users. Besides its diverse utilisation, Twitter is especially known for a platform for political discussion. Both politicians and society use Twitter as a channel to take positions in politcal debates and express opinion. 

While this is desirable and embracing the idea of free-speech on the internet, the question of whether or not Twitter should use tools to automatically detect unwanted content from its platform, such as racism, sexism, false information or hatespeech, is a subject of on-going public debate. 

This project, in a first instance, sets aside the ethical challenges and questions arising, and solely focuses on the technical details of how such a solution might work. The goal of this project is to optimise a machine learning model to automatically detect unwanted content.

## Running this Notebook
---
This notebook contains all code to reproduce the findings of the project as can be seen on the [GitHub](https://github.com/jonas-mika/fyp2021p04g09) page of this project. In order to read in the data correctly, the global paths configured in the section `Constants` need to be correct. The following file structure - as prepared in the `submission.zip` - was followed throughout the project and is recommended to use (alternatively the paths in the section `Constants` can be adjusted):

```
*project tree structure*
```
*Note that the rest of the file structure as can be seen on the [GitHub](https://github.com/jonas-mika/fyp2021p03g09) page of the project generates automatically*

## Required Libraries and Further Imports
---
Throughout the project, we will use a range of both built-in and external Python libraries. This notebook will only run if all libraries and modules are correctly installed on your local machines. 
To install missing packages use `pip install <package_name>` (PIP (Python Package Index) is the central package management system, read more [here](https://pypi.org/project/pip/)). 

In case you desire further information about the used packages, click the following links to find detailed documentations:
- [Pandas](https://pandas.pydata.org/)
- [Numpy](https://numpy.org/)
- [Matplotlib](https://matplotlib.org/stable/index.html)
- [SciKit Learn](https://scikit-learn.org/stable/)
- [NLTK](https://www.nltk.org/)

In [ ]:
%%capture
# uncomment lines with uninstalled packages

#!pip install -U numpy pandas matplotlib seaborn skikit-learn 
!pip install pycontractions
!pip install imblearn
!pip install statsmodels

In [ ]:
# python standard libraries
import json                                            # data transfer to and from json format
import os                                              # access operating system from python
import math                                            # mathematical operations in python
import random                                          # creates randomness
import re                                              # regex search in python
import shutil                                          # system control in python
import warnings                                        # ignore annoying warnings
warnings.filterwarnings("ignore")

# external libraries
import numpy as np                                     # used for numerical calculations and fast array manipulations
import pandas as pd                                    # provides major datastructure pd.DataFrame() to store datasets
import matplotlib
import matplotlib.pyplot as plt                        # basic data visualisation
from matplotlib.ticker import MaxNLocator
import seaborn as sns                                  # advanced data visualisation
from nltk.tokenize import TweetTokenizer               # tokeniser api
from nltk.corpus import stopwords
nltk.download('stopwords');
from pycontractions import Contractions                # intelligently expands contractions in natural language
from collections import Counter                        # counts objects

In [ ]:
# feature extraction
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer # not used yet

# addressing imbalance in data
from imblearn.over_sampling import RandomOverSampler as ROS

# classifiers
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

# performance evaluation
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
print(f'Numpy Version: {np.__version__}')
print(f'Pandas Version: {pd.__version__}')
print(f'Matplotlib Version: {matplotlib.__version__}')
print(f'Seaborn Version: {sns.__version__}')

## Constants
---
To enhance readibilty, as well as to decrease the maintenance effort, it is useful for bigger projects to define contants that need to be accessed globally throughout the whole notebook in advance. 
The following cell contains all of those global constants. By convention, we write them in caps (https://www.python.org/dev/peps/pep-0008/#constants)

In [ ]:
PREPROCESS_DATA = False
GENERATE_IAA_SAMPLE = False

In [ ]:
random.seed(1)

In [ ]:
DATASETS = ['hate', 'emotion']

# store paths
PATH = {}
PATH['data'] = {}
PATH['data']['raw'] = "../data/raw/"
PATH['data']['processed'] = "../data/processed/"

# store data 
DATA = {}
DATA['raw'] = {}
DATA['processed'] = {}
for dataset in DATASETS:
    DATA['raw'][dataset] = {}
    DATA['processed'][dataset] = {}

CORPUS = {}
VOCABULARY = {}

# Fetching Data
---
*TASK 0*

After having setup the overall scheme of the project, we need to fetch the data we want to work on. The datasets were obtained from the
[TweetEval](https://github.com/cardiffnlp/tweeteval#evaluating-your-system) GitHub repository, which provides ready-to-work on tweets and gold-standard annotations for different focuses of analysis - already split up into training, validation and testing sets.

1. Binary Classifcation: **Hate-Speech**
 > Two Labels: *Hate-Speech*/ *Not Hate-Speech*

 > [Raw Data (GitHub)](https://github.com/cardiffnlp/tweeteval/tree/main/datasets/hate)

2. Multiclass Classification: **Emotion Recognition**  
 >Four Labels: *Anger*, *Joy*, *Sadness*, *Optimism* 

 >[Raw Data (GitHub)](https://github.com/cardiffnlp/tweeteval/tree/main/datasets/emotion)

*Note, in order to fetch this data into the Jupyter, the above-mentioned files need to be in the existent in the file structure and the location specified in the code and file tree structure in the introductory section. This should be the case in the submission and if this project was pulled or forked directly from [GitHub](https://github.com/jonas-mika/fyp2021p04g09)* 

## Loading in Data
---
We start off by loading in the data obtained from the above-mentioned sources into the script. All tweets are being read into a list of strings, where each string is representing a single tweet. The golden labels are being read into an index-corresponding array of integers, where the integer at *i*th position is the gold label for the tweet in the list of tweets at position *i*. Lastly, we read in the mapping, between the integer and the corresonding label into a dictionary, such that we can use it for nice plotting.

In [ ]:
# helper function to read in raw files correctly
def read_raw_input(dataset):
    # reading in all .txts into list of strings
    for _file in os.listdir(f'../data/raw/{dataset}'):
        with open(f'../data/raw/{dataset}/{_file}', 'r', encoding='UTF-8') as infile:
            DATA['raw'][dataset][_file[:-4]] = [line.strip() for line in infile.readlines()]

    # convert target labels to integers
    for key in ['train_labels', 'val_labels', 'test_labels']:
        DATA['raw'][dataset][key] = [int(x) for x in DATA['raw'][dataset][key]]

    # convert mapping to dictionary
    DATA['raw'][dataset]['mapping'] = {int(string.split('\t')[0]): string.split('\t')[1] for string in DATA['raw'][dataset]['mapping']}

In [ ]:
# read in the the raw data as specified above for both the hate and emotion data
for dataset in DATASETS:
    read_raw_input(dataset)

## Exploring Data
---
Let's get a feeling for what kind of data we are dealing with. For now, we simply peek into our actual data, the tweets, and output them with their corresponding gold label. After that, we plot the distribution of the labels, which is an important characteristic of the training process, since it might reveal possible imbalances that need to be addressed before training a model.

### Data Size
---
Before starting to tackle the specific data, we should get a feeling with what size of data we are dealing with. Especially in NLP this is an important step, since NLP models usually incorporate high amounts of data in order to build well-performing models.

In [ ]:
!ls -lh ../data/raw/hate

In [ ]:
!ls -lh ../data/raw/emotion

As can be seen, the file sizes are considerably small for NLP datasets. This means, that we most likely will be able to work with the data on local memory. However, we are trying to follow best practice and work RAM efficient during the project.

In [ ]:
!wc -l ../data/raw/hate/*_text.txt

In [ ]:
!wc -l ../data/raw/emotion/*_text.txt

The `hate` dataset contains more than double the tweets than the `emotion` dataset. However, the split is rather similar in both instances with a roughly 70-20-10 split for training, testing and validation.

### Visualising Raw Data
---

In [ ]:
# hate
for i in range(10):
    print(f"{i+1}\tLabel: {DATA['raw']['hate']['mapping'][DATA['raw']['hate']['train_labels'][i]].title()}\t\t{DATA['raw']['hate']['train_text'][i]}")

The first ten tweets from the training set of the hatespeech task look as expected. Tweets are a single-line string, including emojis, syntax, grammar and spelling of the orginal tweet. Both the author of the tweet and links to other users, however, are anonymised for data privacy reasons. 

The gold standard labels appear reasonable for the first ten tweets.

In [ ]:
# emotion
for i in range(10):
    print(f"{i+1}\tLabel: {DATA['raw']['emotion']['mapping'][DATA['raw']['emotion']['train_labels'][i]].title()}\t\t{DATA['raw']['emotion']['train_text'][i]}")

The data for the emotion recognition looks similar. Again, tweets are a single-line string, including emojis, syntax, grammar and spelling of the orginal tweet. Both the author of the tweet and links to other users are anonymised for data privacy reasons. 

Also here, the gold standard labels appear reasonable for the first ten tweets.

### Visualising Label Distribution
---
Visualising the distribution of the target variable in any classification task is an important first step in order to evluate the quality of the data we are training on. Ideally, we would like to observe equally balanced labels, since this prevents models from being overly biased by always predicting a dominant labels. 

In [ ]:
# helper function to plot the distribution of labels in all splits of the data
def visualise_label_distribution(dataset):
    # initalise figure with three axes
    fig, ax = plt.subplots(ncols=3, figsize=(12,4))
    fig.suptitle(f'Frequency of Target Label in {dataset.capitalize()}', fontsize=12, fontweight='bold') # global title

    # plot barplot of label distribution with some additional aesthetic adjustment in each split of the data
    for i, key in enumerate(['train_labels', 'val_labels', 'test_labels']):
        label, count = np.unique(DATA['raw'][dataset][key], return_counts=True)
        ax[i].bar(label, count, color='grey');
        ax[i].set_title(key.replace('_', ' ').title())
        ax[i].set_xticks(label); ax[i].set_xticklabels([string.title() for string in DATA['raw'][dataset]['mapping'].values()])

In [ ]:
visualise_label_distribution(dataset='hate')

The binary label *Hate* (42%) or *Non-Hate* (58%) seems reasonably balanced. Here, balancing of the labels before building the model is possible, but not mandatory. It furthermore becomes obvious that the label ratio is consistent across all splits (training, validation and testing set).

In [ ]:
visualise_label_distribution(dataset='emotion')

However, in the multi-class classifcation task we observe a high imbalance. With only 9% of the total amount of data in each split, the label *optimism* is underrepresent, while the label *anger* (43%) is overrepresented. This imbalance needs to be addressed before training our classifiers. 
Again, however, we observe a consistent label ratio across all splits. 

# Processing of Language Data
---
*TASK 1*

Natural language is a highly-complex type of data, that needs to be processed before it can be implented meaningfully into a classification task. For natural language classifiers the main step of processing the raw data - the original tweets - is called **tokenisation**.

On a high level, computers stand no chance in understanding language. After all, each and every tweet we are dealing with is just a unique concatenation of 0's and 1's, that in itself is useless for classifying the tweet in any way. We must therefore transform our data in a way that our computer is able to understand it. This is done during the process of tokenisation in natural language processing. Tokenisation in NLP generally means the concept of breaking down the original data into smaller parts, called tokens. Those tokens could be sentences, one or more words, or even smaller chunks of text. Either way, they are used to built a vocabulary that ultimately results in a count vector machine, that serves as input to the machine learning model.

## Building Tokeniser
---
In this project we chose to build a word-tokeniser, meaning that the ideal tokeniser recognises words and splits accordingly. However, due to the fact that we are working with tweet data, there are some special requirements in order to get optimal performance in our models. 

**Features of Tokeniser**

1. Lowercase ()

2. Links (Links are a core part of a tweet containing it - if not the core. To not lose information, we must include the links in our classification. However, for the classfication the actual link address is of minor importance. For this reason, we would like to recognise any link, then replace it with some dummy string that the model uses to classify any kind of link in a tweet.

3. Emojis (Emojis have developed to a central method of communicating emotions. It is therefore important that our tokeniser recognises single emojis and feeds them into our classifier.)

4. Hashtags ()

4. Contractions (Has not or Hasn't? It means the same, so we would like our model to recognise the two as the same token.)


The following code cells contain how the tokeniser was built in this project. It is fundamentally based on the *Tweet Tokeniser* provided by the NLTK library and then adds additional functionality using both RegEx and another extyernally trained model for tokenising contractions correctly.

In [ ]:
# loading pre-trained contraction model
if PREPROCESS_DATA == True:
    cont.load_models() # doesn't work on deepnote, because of memory issues. works locally

cont = Contractions(api_key="glove-twitter-200")

In [ ]:
# define url regex pattern and replace pattern globally (compile for speedup)
url_pattern = re.compile(r"(?i)((?:https?://|www\d{0,3}[.]|[a-z0-9.-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|(([^\s()<>]+|(([^\s()<>]+)))))+(?:(([^\s()<>]+|(([^\s()<>]+))))|[^\s`!()[]{};:'\".,<>?«»\“\”‘’]))")
dummy = " <=LINK=> "

# brush up functions
def remove_links(tweet):
    return re.sub(url_pattern, dummy, tweet) # substituting dummy for every link

def remove_newline(tweet):
    return re.sub(r'\\n', ' ', tweet) # deals with issue of newline characters within tweets

def expand(tweet):
    return list(cont.expand_texts([tweet]))[0] # expands contractions

In [ ]:
tk = TweetTokenizer()

def tokenise(tweet):
    cleaned_tweet = expand(remove_links(remove_newline(tweet.lower())))
    return tk.tokenize(cleaned_tweet)

## Exporting Tokenised Tweets
---
We have built the final tokeniser that will be used throughout this project. To not stretch out the RAM, we iterate over the raw data tweet by tweet, tokenise it and write it into an external file. The final tokenised tweets will be separated by tabs, such that they can easily be read back in again, while keeping the split between tokens. 

For both datasets, all processed files will be in saved into a subfolder called `hate` or `emotion` within the folder `processed` in `data`. While exporting, we maintain the naming convention of the raw data. This enables us to use the same function to read in the processed data again. 

*Note that directories are created automatically and thus do not need to be created by hand.*

In [ ]:
if PREPROCESS_DATA == True:
    for dataset in DATASETS:
        # create preprocessed folder
        try: os.makedirs(f'../data/processed/{dataset}')
        except: None

        # preprocess and tokenize tweets
        for key in ['train_text', 'val_text', 'test_text']:
            with open(f'../data/processed/{dataset}/{key}.txt', 'w', encoding = 'UTF-8') as outfile:
                for tweet in DATA['raw'][dataset][key]:
                    outfile.write('\t'.join(tokenise(tweet)) + '\n') # writing tokenised tweet with tab delimiter

        # copy labels and mapping without preprocessing
        for key in ['train_labels', 'val_labels', 'test_labels', 'mapping']:
            shutil.copyfile(f'../data/raw/{dataset}/{key}.txt', f'../data/processed/{dataset}/{key}.txt')

In [ ]:
# loading in processed data
for dataset in DATASETS:
    for key in os.listdir(f'../data/processed/{dataset}'):
        with open(f'../data/processed/{dataset}/{key}', 'r', encoding='UTF-8') as infile:
            DATA['processed'][dataset][key[:-4]] = [line.strip().split('\t') for line in infile.readlines()]

    # convert target labels to integers
    for key in ['train_labels', 'val_labels', 'test_labels']:
        DATA['processed'][dataset][key] = [int(x) for x in [DATA['processed'][dataset][key][i][0] for i in range(len(DATA['processed'][dataset][key]))]]

    # convert mapping to dictionary
    DATA['processed'][dataset]['mapping'] = {int(string[0]): string[1] for string in DATA['processed'][dataset]['mapping']}

## Testing Tokeniser
---
The tokeniser is built and has been applied onto the entire corpus. Let's evaluate its performance using the same first ten tweets as we peeked into in the first section.

In [ ]:
for i in range(10):
    print(f"Original Tweet: {DATA['raw']['hate']['train_text'][i]}\nTokenised Tweet: {'-'.join(DATA['processed']['hate']['train_text'][i])}\n")

In [ ]:
for i in range(10):
    print(f"Original Tweet: {DATA['raw']['emotion']['train_text'][i]}\nTokenised Tweet: {'-'.join(DATA['processed']['emotion']['train_text'][i])}\n")

In both small samples from the two datasets the tokeniser seems to perform well. As expected, we split the tweets by words. Furthermore, the tokeniser is able to keep hashtags, expand contractions, deal correctly with emojis and replace links with the dummy string.
We can now continue our work based on the tokenised data.

# Descriptive Statistics
---
*TASK 2* 

An essential part of any NLP classfication task is to describe the corpus, so the entire tokenised data. In this section, we will step-by-step compute the most important statistics and visualise some of them for both datasets.

## Corpus
---
Generally in linguistics, a corpus is a large and structured set of texts. The corpus is often used as the basis for a lot of descriptive analysis of the language data at hand. We therefore create a corpus for both training sets of our datasets using a flattened list, that is a linear conctentation of all tokens in the training sets of both datasets.

In [ ]:
# build entire corpus from training set in both datasets
for dataset in DATASETS:
    CORPUS[dataset] = [token for tweet in DATA['processed'][dataset]['train_text'] for token in tweet]

In [ ]:
for dataset in DATASETS:
    print(f'Corpus Size ({dataset.title()}): {len(CORPUS[dataset])}')

As we can see, the corpus for the `hate` classification is significantly larger with 216.076 total number of tokens than the `emotion` corpus with 60.506 tokens.

## Vocabulary 
---
In NLP, the set of unique words used in the text corpus is referred to as the vocabulary. The vocabulary is the basis for training machine learning models based on the bag-of-words approach, which treats every unique token as a feature in the classification.

In order to built the vocabulary, we use the `Counter` class from the `collections` standard library in python onto the corpus, in order to compute a dictionary containing all unique tokens (types) together with their corresponding frequency. We combine all data, together with some additional information like the `rank`, where 1 is the the most frequent token, the `normalised/ relative frequency` (frequency divided by total number of tokens) and the `cumulative frequency`. The final data frame will be sorted in descending order, from most to least frequent tokens.

In [ ]:
# build entire vocabulary from training tweets in both datasets
for dataset in DATASETS:
    corpus = CORPUS[dataset]

    # initialise frequency counter
    VOCABULARY[dataset] = pd.DataFrame.from_dict(Counter(corpus), orient='index').reset_index().rename(columns={'index': 'token', 0: 'frequency'}).sort_values(by='frequency', ascending=False).reset_index().drop(columns=['index'])

    VOCABULARY[dataset]['rank'] = VOCABULARY[dataset].index + 1 # add one to index to build rank
    VOCABULARY[dataset]['normalised_frequency'] = VOCABULARY[dataset]['frequency'] / VOCABULARY[dataset]['frequency'].sum() # relative frequency
    VOCABULARY[dataset]['cumulative_frequency'] = VOCABULARY[dataset]['normalised_frequency'].cumsum()

Let's have a look at the vocabulary dataframes constructed for both dataframes.

In [ ]:
VOCABULARY['hate']

In [ ]:
VOCABULARY['emotion']

## Type/ Token Ratio
---
The type/ token ratio of is the quotient of number of types (vocabulary size) to number of tokens (text/corpus size). We compute it in the following way. 

In [ ]:
for dataset in DATASETS:
    print(f'TTR ({dataset.title()}): {round(len(VOCABULARY[dataset]) / len(CORPUS[dataset]),2)}')

A high TTR generally indicates a high degree of lexical variation in the corpus. However, in this context the higher TTR in the emotion dataset might also result from the fact that it is a lot smaller in size than the hate dataset and thus the TTR score is less biased by few very frequently occurring tokens that decrease the TTR score.

## Zipf's Law
---
Zipf's Law says that the frequency of a word is inversely proportional to its rank in the frequency table. Thus, very few words occur very frequently, while the majority of words have very low frequencies. Tyical frequent words are function words, such as prepositions, pronouns, conjunctions and similar words.

Let's visualise Zipf's Law for both of our datasets.

In [ ]:
def visualise_zipf(dataset, log = False):
    if log:
        x = np.log(VOCABULARY[dataset]['rank'])
        y = np.log(VOCABULARY[dataset]['frequency'])
    else: 
        x = VOCABULARY[dataset]['rank']
        y = VOCABULARY[dataset]['frequency']

    fig, ax = plt.subplots(ncols=2, figsize=(15,6))
    ax[0].plot(x, y);
    ax[0].set_xlabel('Rank'); ax[0].set_ylabel('Frequency'); ax[0].set_title(f'Frequency of Tokens in {dataset.title()}'); 
    ax[1].plot(VOCABULARY[dataset]['rank'], VOCABULARY[dataset]['cumulative_frequency']);
    ax[1].set_xlabel('Rank'); ax[1].set_ylabel('Cumulative Frequency'); ax[1].set_title(f'Cumulative Frequency of Tokens in {dataset.title()}'); 

In [ ]:
visualise_zipf(dataset='hate', log=False)

In [ ]:
visualise_zipf(dataset = 'emotion', log = False)

# Digression: N-Gram Model
---
N-Gram Language models are an important tool in NLP, i.e. used in the context of auto-completion or tools like spell checkers. The idea of the model is rather simple: Given a sequence of words, predict the next word that appears with highest probability. In that sense, N-Gram models are a way of finding the maximum likelihood word after a sequence of previous words. But, how do we find this maximum likelihood word? From the point of probability theory, the probability of a word occuring is the conditional probability of seeing some word, given that we already know that that a sequence of previous words has occurred. However, calculating those probabilites for bigger coropora is computationally infeasible. 

However, one can simplify the computation. The *k*th order *Markov assumption* states that each element of a sequence of tokens only depends on the *k* immediately preceding elements, such that manual 

$$
p(w_i|w_{1}, ...,w_{i-1}) \approx (w_i|w_{i-k}, ..., w_{i-1})
$$



*Note: For exploring n-gram models we are using the training set of the `hate` dataset.*

## Manual Model Prediction
---
Before using external libraries to implement n-gram models, we wanted to give it a shot and code it out for ourselves in the simplest of all cases, the bigram. In this simple model we iterate over each tokenised tweet in the data pairwise, and while doing so incrementing the counter of seeing word 2, given that we have seen word 1 in a nested dictionary. We can then easily convert those counts into conditional probability and from that choose the word that is maximally likely give some word. A `generate` function is then the natrual extension that consecutively chooses the next most likely word for a sequence of *n* words.

In [ ]:
from nltk import bigrams
from collections import Counter, defaultdict

class OurBigram:
    def __init__(self):
        # initialise empty skeleton structure of model as nested dictionary (initialise all counts to 0)
        self.model = defaultdict(lambda: defaultdict(lambda: 0))
    
    def fit(self, data):
        # count frequency of co-occurances
        for sentence in data:
            for w1, w2 in bigrams(sentence, pad_right=True, pad_left=True):
                self.model[w1][w2] += 1
            
        # transform counts to conditional probabilities
        for w1 in self.model:
            total_count = float(sum(self.model[w1].values()))
            for w2 in self.model[w1]:
                self.model[w1][w2] /= total_count

    def find_mle(self, word):
        # for a given word return the word that occurs next with maximum likelihood
        return max(self.model[word], key=self.model[word].get)
            
    def generate(self, word, n):
        # generate a sequence of n words given an initial word from the bigram model
        while n >= 0:
            print(word, end=' ') 
            mle = self.find_mle(word)
            word = mle
            n-=1


bigram = OurBigram()
bigram.fit(DATA['processed']['hate']['train_text'])
bigram.generate('hey', 4)

## NLTK Language Model
---
In this section, we are exploring externally implemented language models from the NLTK library that we assume ot perform overall better, i.e. in the way it deals with unseen tokens. The following lines of code show, how the model is being trained on hte training set of the `hate` data. We finally generate a sample tweet of length ten.

In [ ]:
# imports for n-gram language model
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE

In [ ]:
# number of grams
n=5

# split 
train, vocab = padded_everygram_pipeline(n, DATA['processed']["hate"]["train_text"])

# initialise and fit data
lm = MLE(n)
lm.fit(train, vocab)

In [ ]:
'hitler ' + ' '.join(lm.generate(10, text_seed=['hitler'], random_seed=10))

We can see that the model is capable of occational, syntactically correct and plausible, hate speech.

# Digression: Manual Annotations and Inter-Annotator Agreement (IAA)
---
*TASK 3*

Classification tasks in ML are concerned with predicting a class membership of some object based on computed features. Just as with any other type of objects of data we are dealing with, there are different ways of predicting these labels. 

1. **Explicit Rules**: Defining explicit rules in what instances of features, what label to predict

2. **Unsupervised Learning**: Letting the machine find clusters within the data by itself

3. **Supervised Learning**: Manually enriching the data with additional information through annotation.

Defining explicit decision rules is most of the times infeasible for more complex classfication problems. Likewise, unsupervised learning models in NLP often do not give the wanted results. For this reason, the predominant method for knowledge encoding when dealing with natural language is the supervised learning that involves the process of manual annotation. 

Manually annotating data means to define some kind of *gold standard*, which the label our model ideally predicts. However, this in itself can be a challenge, since language can be ambigious, and interpretation is often subject of personal opinion. 

In this section we therefore manually annotate a small percentage (100 tweets) of the training corpus in the `hate` dataset and compute metrics to what degree we agreed on class membership. We followed the [Annotation Guidelines](https://github.com/msang/hateval/blob/master/annotation_guidelines.md) as employed by the source of the data ([Final Paper](https://www.aclweb.org/anthology/S19-2007.pdf))


## Generate IAA Sample
---
We first generate the sample of tweets and corresponding gold standard labels as reported by the source of the data. For ease, we chose the last 100 tweets from the training corpus of the `hate` dataset. The tweets were exported into a separate file in the directory `data/annotations`.

In [ ]:
if GENERATE_IAA_SAMPLE == True:
    with open('../data/annotations/annotation_sample.txt', 'w') as outfile:
        for tweet in DATA['raw']['hate']['train_text'][-100:]: # last 100 tweets in hate training set
            outfile.writelines(tweet + '\n')
    with open('../data/annotations/ground_truth_annotation_sample.txt', 'w') as outfile:
        for label in DATA['raw']['hate']['train_labels'][-100:]:
            outfile.writelines(str(label) + '\n')

Without looking at the gold standard labels from the source of the data, each member of the group individually annotated the 100 tweets and uploaded the file into the folder `data/annotations/manual_annotations`. The following section evaluates the results of this annotation.

## Evaluation
---
There exist different metrics for evaluating annotations of multiple annotators, which we will compute within this section. For easy dealing, we read in the manual annotations and load them into a single dataframe, containing both the original tweet, the gold standard label and the four annotated labels by each group member.

In [ ]:
# compare manual annotations through building dataframe
annotation_evaluation = pd.DataFrame({'tweets': DATA['raw']['hate']['train_text'][-100:], 'ground_truth': DATA['raw']['hate']['train_labels'][-100:]})

for filename in os.listdir('../data/annotations/manual_annotations'): # opening manually annotated files
    with open(f'../data/annotations/manual_annotations/{filename}', 'r') as infile:
        annotation_evaluation[filename[:-4]] = [int(line.strip()) for line in infile.readlines()]

In [ ]:
annotation_evaluation

### Classification Report 
---
One way of evaluating the annotation is to treat each annotator as a machine learning model that predicted some label. Then comparing the predicted labels with the set gold-standard allows us to compute common evaluation metrics, such as accuarcy, recall and sensitivity and f-scores, in order to evaluate the quality of the annotations in relation to the assumed gold standard.

#### Nicola

In [ ]:
print('#'*23 + ' ' + 'Nicola' + ' ' + '#'*23 + '\n')    
print(pd.DataFrame(confusion_matrix(annotation_evaluation['ground_truth'], annotation_evaluation['nicola_annotations']), columns=['Predicted Non-Hate', 'Predicted Hate'], index=['Non-Hate', 'Hate']))
print('\n')
print(classification_report(annotation_evaluation['ground_truth'], annotation_evaluation['nicola_annotations'], target_names=["Non-Hate","Hate"]))

#### Hugo

In [ ]:
print('#'*23 + ' ' + 'Hugo' + ' ' + '#'*23 + '\n')    
print(pd.DataFrame(confusion_matrix(annotation_evaluation['ground_truth'], annotation_evaluation['hugo_annotations']), columns=['Predicted Non-Hate', 'Predicted Hate'], index=['Non-Hate', 'Hate']))
print('\n')
print(classification_report(annotation_evaluation['ground_truth'], annotation_evaluation['hugo_annotations'], target_names=["Non-Hate","Hate"]))

#### Mika

In [ ]:
print('#'*23 + ' ' + 'Mika' + ' ' + '#'*23 + '\n')    
print(pd.DataFrame(confusion_matrix(annotation_evaluation['ground_truth'], annotation_evaluation['mika_annotations']), columns=['Predicted Non-Hate', 'Predicted Hate'], index=['Non-Hate', 'Hate']))
print('\n')
print(classification_report(annotation_evaluation['ground_truth'], annotation_evaluation['mika_annotations'], target_names=["Non-Hate","Hate"]))

#### Aidan

In [ ]:
print('#'*23 + ' ' + 'Aidan' + ' ' + '#'*23 + '\n')    
print(pd.DataFrame(confusion_matrix(annotation_evaluation['ground_truth'], annotation_evaluation['aidan_annotations']), columns=['Predicted Non-Hate', 'Predicted Hate'], index=['Non-Hate', 'Hate']))
print('\n')
print(classification_report(annotation_evaluation['ground_truth'], annotation_evaluation['aidan_annotations'], target_names=["Non-Hate","Hate"]))

In summary, the confusion matrices and classification reports show that the even in a relatively small sample of only 100 tweets and only four different annotators, hatespeech seems to be perceived differently. Within the group, Nicola was most sensible towards hate-speech, meaning that out of all she labelled the most tweets as being hate speech. This resulted in a high recall score on the hate data and an overall balanced prediction. Hugo, in turn only tagged 15 tweets as being hate speech. His insensitivy towards labelling hatespeech resulted in a high perfect recall score on non-hate and perfect precision on hate, i.e. whenever he labelled something as hatespeech, there is absolutely no doubt that it is actually hatespeech. However, he also falsely predicted a lot of hate-tweets as being non-hate, resulting in a poor recall score on hate tweets. 

The reports show that annotation of language data with regards to the general *state-of-mind* of the author is subject to personal perception. Thus, we have to find ways of agreeing on some gold-standard. The following chapter explores how we can numerically summarise annotations of multiple annotators to agree on target labels.

### Cohen's Kappa
---
Cohen's Kappa ($\kappa$) is an evaluation metric that is used to measure inter-rater reliability for labelling (categorical) items. In NLP it is i.e. used to compare the annotations of a single annotator with the original annotations. In general, Cohen's Kappa is preferred over simple percentage of agreement, since it corrects the score by the agreement that would be expected by pure chance. 
Cohen's Kappa values range from  from in the interval $\kappa \in [-1,1]$, with larger values indicating higher agreement between two classifications. 

The following code cell computes the Cohen's Kappa score for each annotator using the function `cohen_kappa_score` from `sci-kit learn`.

In [ ]:
from sklearn.metrics import cohen_kappa_score

# compute cohen's kappa score for each annotator
for person in list(annotation_evaluation)[-4:]:
    print(f'{person[:-12].title()}:\t {round(cohen_kappa_score(annotation_evaluation["ground_truth"], annotation_evaluation[person]),2)}')

The Cohen Kappa scores of the group's annotations suggest a *substantial agreement* with the original labelling, as three out of four of the scores are above 0.61. Hugo's score is less but is still considered *fair*.

One can summarise, that from the Cohen Kappa scores there is no substantial evidence against the original labelling being invalid.

### Fleiss Kappa
---
Fleiss' Kappa is a statistical measure for assessing the reliability of agreement between a fixed number of annotators when labels to a number of classifying items. In contrast to Cohen's Kappa it is metric between multiple annotators and is not only 

The measure calculates the degree of agreement in classification over that which would be expected by chance.

In [ ]:
from statsmodels.stats.inter_rater import fleiss_kappa

# fleiss kappa with ground_truth
annotation_list = []
for row_index in range(len(annotation_evaluation)):
    zero = len([elm for elm in annotation_evaluation.iloc[row_index, 1:] if elm == 0])
    annotation_list.append([zero, 5-zero])

annotation_table = np.array(annotation_list)

print("Fleiss's K (Including Ground Truth):", round(fleiss_kappa(annotation_table),2))

# fleiss kappa without ground_truth
annotation_list = []
for row_index in range(len(annotation_evaluation)):
    zero = len([elm for elm in annotation_evaluation.iloc[row_index, 2:] if elm == 0])
    annotation_list.append([zero, 4-zero])

annotation_table = np.array(annotation_list)

print("Fleiss's K (Excluding Ground Truth):", round(fleiss_kappa(annotation_table),2))

In both cases, the Fleiss Kappa score can be interpreted as a *moderate agreement*.

The fact that the kappa score is higher with the ground truth labels being included tells us that the the 3/4 people who are more in agreement with eachother are more in agreement with the original labelling than Hugo's. 

# Classification
---
*TASK 4*

In [ ]:
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV

def build_model(dataset):
    # combining train and validation, because grid search cross validates to find best params
    X_train = DATA['processed'][dataset]['train_text'] + DATA['processed'][dataset]['val_text']
    y_train = DATA['processed'][dataset]['train_labels'] + DATA['processed'][dataset]['val_labels']

    X_test, y_test = DATA['processed'][dataset]['test_text'], DATA['processed'][dataset]['test_labels']

    # building pipeline to build classifier
    # a) vectorise tokenised tweets into sparse matrix 
    # b) term frequency inverse document frequency for better performance
    # c) support vector machine model
    classifier = Pipeline([('vect', CountVectorizer(preprocessor=lambda x:x, tokenizer=lambda x:x, stop_words=stopwords.words('english'))),
                           ('ovs', SMOTE()),
                           ('tfidf', TfidfTransformer()),
                           ('clf', SGDClassifier())])

    # hyperparameters to tune
    parameters = {
        'vect__max_df': (0.5, 0.7, 0.9),
        'vect__max_features': (2000, 5000),
        'vect__stop_words': [None, 'english'],
        #'tfidf__use_idf': (True, False),
        #'tfidf__norm': ('l1', 'l2'),
        'clf__loss': ['hinge', 'log'],
        #'clf__alpha': (0.00001, 1e-3),
        #'clf__penalty': ('l2', 'elasticnet'),
    }

    grid = GridSearchCV(classifier, param_grid = parameters, n_jobs=-1, scoring='accuracy')
    grid = grid.fit(X_train, y_train) # fit

    print('Found Optimal Parameters!')
    print(f'Parameters: {grid.best_params_}')
    print(f'Best Macro F1-Score: {grid.best_score_}')

    return grid

## Binary Classification
---

In [ ]:
hate_model = build_model(dataset = 'hate')

In [ ]:
# evaluate
X_test, y_test = DATA['processed']['hate']['test_text'], DATA['processed']['hate']['test_labels']

predictions = hate_model.predict(X_test)
print(classification_report(y_test, predictions))

## Multiclass Classification
---

In [ ]:
emotion_model = build_model(dataset = 'emotion')

In [ ]:
# evaluate
X_test, y_test = DATA['processed']['emotion']['test_text'], DATA['processed']['emotion']['test_labels']

predictions = emotion_model.predict(X_test)
print(classification_report(y_test, predictions))

In [ ]:
######################################################## NICOLA

## Helper Functions
---

In [ ]:
def remove_stop_uncommon(tokenised, vocab, min_frequency, remove_stop=False):
    """removes stop words and words that occur less than the min frequency in the training data, from tokenised
    inputs:tokenised: tokenised data, min_frequency: integer indicating min frequency of word to be kept,vocab: the vocab of the training data,remove_stop:True/False"""
    
    # initialising set of stop words
    stop_words = set()
    if remove_stop == True:
        stop_words = set(stopwords.words('english')) #set to downloaded stop word list

    #creating helper dictionary of with key being uniques words and value being the frequency
    vocab_dict = {vocab['token'][x]:vocab['frequency'][x] for x in range(vocab.shape[0])} 
    
    #for token_list in tokenised: #if not in the temp vocab dict then append as 0 to avoid key error
    #    for token in token_list:
    #        try: vocab_dict[token] = vocab_dict[token]
    #        except: vocab_dict[token] = 0

    return [[token for token in tweet if token not in stop_words and vocab_dict[token] >= min_frequency] for tweet in tokenised] #list comprehention that returns a tokenised version with the correct words removed(depending on parameters chosen)

In [ ]:
def dummy(doc):
    """does nothing"""
    return doc
    
def vectorise(dataset, tokenised, tvt="val"):
    """formats the data into a format that is readable by the models, a sparce matrix
    input: dataset should be either "hate" or "emotion", tokenised should be tokenised data. tvt should be "train" if formatting training data """
    
    # initialising count vectoriser without further parameters and fitting data
    cv = CountVectorizer() 
    cv.fit(DATA["processed"][dataset]["train_text"]) # fit to training text(creates empty sparce matrix)

    if tvt=="train": #creates the values in the sparce matrix for the relevant data 
        X_train_counts = cv.transform(DATA["processed"][dataset]["train_text"])
    else:
        X_train_counts = cv.transform(tokenised)        
    return X_train_counts #returns sparce matrix

In [ ]:
ros_ = ROS(random_state=123)
def fit_model(Classifier,Dataset,min_frequency=1,balance=True,remove_stop_=True):
    """Creates a model
    Inputs:Classifier:"RFC"/"MNB"/"Baseline",Dataset:"emotion"/"hate",min_frequency=1 dont think we will use (could remove parameter),balance and reomve_stop are booleans that determine if data is balanced and if stop words are removed"""
    if balance==True:#balancing the data using RandomOverSampler 
        X, y = ros_.fit_resample(text_format (Dataset,
    remove_stop_uncommon(DATA["processed"][Dataset]["train_text"],min_frequency,VOCABULARY[Dataset],remove_stop=remove_stop_),tvt="train"),
    DATA['processed'][Dataset]['train_labels'])
    else:# set to unbalanced data
        X = text_format(Dataset,remove_stop_uncommon(DATA["processed"][Dataset]["train_text"],min_frequency,VOCABULARY[Dataset],remove_stop=True),tvt="train")
        y = DATA['processed'][Dataset]['train_labels']
    if Classifier=="RFC":##build relevant model
        model = RandomForestClassifier(max_depth=None, n_estimators=100,random_state=0)
    elif Classifier=="MNB":
        model=MultinomialNB(alpha=0.)### this alpha value at its defaut may actually give better results
    elif Classifier=="Baseline":
        model = SGDClassifier(loss='log')
    return model.fit(X,y)#fit the model and return

In [ ]:
def model_accuracy_val(min_frequency,Dataset,model,v_or_t="val",remove_stop_=False):
    """takes cleaned and tokensed data and returns accuracy score of the model
    input:min_frequency:int(see remove_stop_uncommon),Dataset:"hate"/"emotion",model:a fitted model,v_or_t:"test" if using test data,remove_stop_:boolean value if val/test should remove stop words"""
    if v_or_t !="val":#warn and set to "test" if v_or_t is changed
        v_or_t="test"
        print("warning! you are using the test data")
    prediction = model.predict(text_format(Dataset,remove_stop_uncommon(DATA["processed"][Dataset][f"{v_or_t}_text"],min_frequency,VOCABULARY[Dataset],remove_stop=remove_stop_),tvt=v_or_t))
    actual = DATA["processed"][Dataset][f"{v_or_t}_labels"]
    return accuracy_score(actual, prediction)#return accuracy score at the given parameters

In [ ]:
def model_prf_score(Dataset,min_frequency,model,v_or_t="val",remove_stop__=False):
    """takes cleaned and tokensed data and returns precision, recall and F-score of the model
    inputs:Dataset:"hate"/"emotion",min_frequency:int(see remove_stop_uncommon),model:a fitted model,v_or_t:"test" if using test data,remove_stop_:boolean value if val/test should remove stop words"""
    if v_or_t !="val":#warn and set to "test" if v_or_t is changed
        v_or_t="test"
        print("warning! you are using the test data")
    prediction = model.predict(text_format(Dataset,remove_stop_uncommon(DATA["processed"][Dataset][f"{v_or_t}_text"],min_frequency,VOCABULARY[Dataset],remove_stop=remove_stop__),tvt=v_or_t))
    actual = DATA["processed"][Dataset][f"{v_or_t}_labels"] 
    scores= precision_recall_fscore_support(actual,prediction)#save scores into scores
    order=list(DATA["raw"][Dataset]["mapping"].values())# this gives the labels for the colums and rows of the dataframe
    score_labels=["Precision","Recall","F-score", "Support"]
    dataframe_of_scores=pd.DataFrame(data=scores,columns=order,index=score_labels)
    list_of_accuracy=[model_accuracy_val(min_frequency,Dataset,model,v_or_t="val",remove_stop_=remove_stop__) for x in range(len(order))]
    dataframe_of_accuracy=pd.DataFrame(data=np.array([list_of_accuracy]), columns=order, index=["Accuracy"])
    #print(dataframe_of_accuracy)

    return pd.concat([dataframe_of_scores,dataframe_of_accuracy])#return pandas df with scores

In [ ]:
def plot_accuracy(max_vocab,Dataset,model,ax,remove_stop__=False):
    """plots onto a subplot: the accuracy score of 1 model given validation data with different min_freq values
    input:max_vocab:int shows accuracy in range [0,max_vocab],Dataset:"hate"/"emotion",model:a fitted model,ax:an axis for the sublot e.g.[0,0],remove_stop__:boolean value if val should remove stop words"""
    accuracies=[]
    for i in range (max_vocab):# for each min_freq=i, find accuracy score and append to accuracies
        accuracies.append(model_accuracy_val(i,Dataset,model,remove_stop_=remove_stop__))
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))#plot integer values on the xaxis
    ax.plot([x for x in range (max_vocab)],accuracies)
    ax.set_xlabel("Minimum Word Frequecy")
    ax.set_ylabel("Accuracy Score")
    ax.set_title("Accuracy and Minimum Word Frequecy in Validation Data")

In [ ]:
def plot_prf_accuracy(Dataset,max_freq,model_list,model_names,v_or_t_="val"):
    scores=["Precision","Recall","F-score","Accuracy"]#list of scores here
    if v_or_t_ !="val":#warn and set to "test" if v_or_t is changed
        v_or_t_="test"
        print("warning! you are using the test data")
    num_labels=len(list(DATA["raw"][Dataset]["mapping"].values()))
    fig,ax=plt.subplots(num_labels,4,figsize=(20,3*num_labels))
    fig.tight_layout(pad=0.4, w_pad=1, h_pad=3.0)
    fig.suptitle(f"PRF and Accuracy Scores for {Dataset} {model_names[-1]} Models",y=1.1,fontsize="x-large")
    for x,model in enumerate(model_list):#each column
        dataframes=[]
        for i in range(max_freq):
            dataframes.append(model_prf_score(Dataset,i,model,v_or_t=v_or_t_))
        for y,label in enumerate(list(DATA["raw"][Dataset]["mapping"].values())):#each row
            ax[y,x].set_title(f"{label.title()} and {model_names[x]}") 
            ax[y,x].set_ylim(0,1)
            ax[y,x].xaxis.set_major_locator(MaxNLocator(integer=True))#plot integer values on the xaxis
            for score in scores:#each line
                y_values=[]
                for i in range(max_freq):
                    value=dataframes[i][label][score]
                    y_values.append(value)
                ax[y,x].plot([x for x in range (max_freq)],y_values,label=score)
    handles, labels = ax[y,x].get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper right',ncol=3,fontsize="small")

### Loading in Models

#### Binary

In [ ]:
Baseline_hate=fit_model("Baseline","hate",balance=False,remove_stop_=False)
Baseline_hate_bal=fit_model("Baseline","hate",remove_stop_=False)
Baseline_hate_stop=fit_model("Baseline","hate",balance=False)
Baseline_hate_bal_stop=fit_model("Baseline","hate" )

In [ ]:
MNB_hate=fit_model("MNB","hate",balance=False,remove_stop_=False)
MNB_hate_bal=fit_model("MNB","hate",remove_stop_=False)
MNB_hate_stop=fit_model("MNB","hate",balance=False)
MNB_hate_bal_stop=fit_model("MNB","hate")

In [ ]:
RFC_hate=fit_model("RFC","hate",balance=False,remove_stop_=False)
RFC_hate_bal=fit_model("RFC","hate",remove_stop_=False)
RFC_hate_stop=fit_model("RFC","hate",balance=False)
RFC_hate_bal_stop=fit_model("RFC","hate")

#### Multiclass

In [ ]:
Baseline_emotion=fit_model("Baseline","emotion",balance=False,remove_stop_=False)
Baseline_emotion_bal=fit_model("Baseline","emotion",remove_stop_=False)
Baseline_emotion_stop=fit_model("Baseline","emotion",balance=False)
Baseline_emotion_bal_stop=fit_model("Baseline","emotion")

In [ ]:
MNB_emotion=fit_model("MNB","emotion",balance=False,remove_stop_=False)
MNB_emotion_bal=fit_model("MNB","emotion",remove_stop_=False)
MNB_emotion_stop=fit_model("MNB","emotion",balance=False)
MNB_emotion_bal_stop=fit_model("MNB","emotion")

In [ ]:
RFC_emotion=fit_model("RFC","emotion",balance=False,remove_stop_=False)
RFC_emotion_bal=fit_model("RFC","emotion",remove_stop_=False)
RFC_emotion_stop=fit_model("RFC","emotion",balance=False)
RFC_emotion_bal_stop=fit_model("RFC","emotion")

### Running Models on Validation Data

#### Binary
Evaluation of performance:<br>
(If F-score is interpreted as most important) <br>
Best Baseline: unclear, however the models trained on unbalanced data should not be used. <br>
Best MNB: unclear, however the models trained on unbalanced data should not be used.  <br>
Best RFC: models trained on balanced data are better.<br>
<br>_
Best Overall: Random Forrest, balanced, stop words irrelevant so remove_stop=False, no imporvement with small increase min freq so min_freq=1 should be used(to be expected with this classifier) <br>

In [ ]:
models=[Baseline_hate,Baseline_hate_bal,Baseline_hate_stop,Baseline_hate_bal_stop]
model_names=["Baseline_hate","Baseline_hate_bal","Baseline_hate_stop","Baseline_hate_bal_stop","Baseline SGD Classifier"]
plot_prf_accuracy("hate",50,models,model_names)

In [ ]:
models=[MNB_hate,MNB_hate_bal,MNB_hate_stop,MNB_hate_bal_stop]
model_names=["MNB_hate","MNB_hate_bal","MNB_hate_stop","MNB_hate_bal_stop","MNB Classifier"]
plot_prf_accuracy("hate",50,models,model_names)

In [ ]:
models=[RFC_hate,RFC_hate_bal,RFC_hate_stop,RFC_hate_bal_stop]
model_names=["RFC_hate","RFC_hate_bal","RFC_hate_stop","RFC_hate_bal_stop","Random Forest Classifier"]
plot_prf_accuracy("hate",50,models,model_names)

#### Mulitclass
Evaluation:<br>
 (If F-score is interpreted as most important) <br>
Best Baseline: <br>
Best MNB: <br>
Best RFC: Balanced with our without stop words<br>

In [ ]:
models=[Baseline_emotion,Baseline_emotion_bal,Baseline_emotion_stop,Baseline_emotion_bal_stop]
model_names=["Baseline_emotion","Baseline_emotion_bal","Baseline_emotion_stop","Baseline_emotion_bal_stop","Baseline SGD Classifier"]
plot_prf_accuracy("emotion",50,models,model_names)

In [ ]:
models=[MNB_emotion,MNB_emotion_bal,MNB_emotion_stop,MNB_emotion_bal_stop]
model_names=["MNB_emotion","MNB_emotion_bal","MNB_emotion_stop","MNB_emotion_bal_stop","MNB Classifier"]
plot_prf_accuracy("emotion",50,models,model_names)

In [ ]:
models=[RFC_emotion,RFC_emotion_bal,RFC_emotion_stop,RFC_emotion_bal_stop]
model_names=["RFC_emotion","RFC_emotion_bal","RFC_emotion_stop","RFC_emotion_bal_stop","Random Forest Classifier"]
plot_prf_accuracy("emotion",50,models,model_names)

#### Results With Test Data

In [ ]:
model_prf_score("hate",1,M_hate_bal)###best scoring validation

In [ ]:
model_prf_score("hate",5,MNB_hate_bal,v_or_t="test")###test data

In [ ]:
models=[RFC_hate,RFC_hate_bal,RFC_hate_stop,RFC_hate_bal_stop]
model_names=["RFC_hate","RFC_hate_bal","RFC_hate_stop","RFC_hate_bal_stop","Random Forest Classifier"]
plot_prf_accuracy("hate",10,models,model_names,v_or_t_="test")

In [ ]:
confusion_matrix(actual,prediction, labels=None, sample_weight=None, normalize=None)

## To Do:
* task 2 descriptive things
* paper 
* make the formatting of the notebook good enough to be readable
* understanding laplace or better estimate instead -smoothing
* improvements to model (4):
- [x] remove min freq
- [x] random forrest
- [x] stopwords (make comparable)
- [x] Multinomial NB model -why and whats different 
* make some graphics to compare values other than accuracy

Nici:
* plot_prf_accuracy comments
* summary of graphs
* running with test data
* graphics



important note: if we add a smooting on the multinomialNB by removing the alpha=0. parmeter then the results are better but different. worth looking into as a group, can ammend functions later to accomodate- nicola

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7ea65742-831a-48cd-9d56-a22e1ed66c7b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>